In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Dict, Tuple, Union, Optional
import sys
from pathlib import Path


# Get the parent directory (Datos folder)
current_dir = Path().resolve()  # libretas folder
datos_dir = current_dir.parent  # Datos folder

# Add Datos to path so we can import from Functions
sys.path.insert(0, str(datos_dir))

pd.set_option("display.max_columns", None)


In [2]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import leaguestandingsv3
from datetime import datetime, timezone, timedelta
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard
from Functions.helpfuncs import get_current_season
from Functions.games import *

In [3]:
eastern_conference = {
    'ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DET', 'IND',
    'MIA', 'MIL', 'NYK', 'ORL', 'PHI', 'TOR', 'WAS'
}

western_conference = {
    'DAL', 'DEN', 'GSW', 'HOU', 'LAC', 'LAL', 'MEM', 'MIN',
    'NOP', 'OKC', 'PHX', 'POR', 'SAC', 'SAS', 'UTA'
}

In [84]:
class Teams():
    def __init__(self, team_abbreviation: str = None, team_id:str = None, team_name:str = None):
        self.current_season = get_current_season()
        provided = [
            team_abbreviation is not None,
            team_id is not None,
            team_name is not None
        ]

        if sum(provided) != 1:
            raise ValueError("You must provide exactly one of: team_abbreviation, team_id, team_name")

        # buscar por el parámetro correcto
        if team_abbreviation is not None:
            self.team_details = teams.find_team_by_abbreviation(team_abbreviation)

        elif team_id is not None:
            self.team_details = self.get_team_details_by_id(team_id)

        elif team_name is not None:
            self.team_details = self.get_team_details_by_name(team_name)

        # añadir conferencia
        if not self.team_details:
            raise ValueError("Team not found with the provided identifier.")
        abb = self.team_details["abbreviation"]
        self.team_details["conference"] = "East" if abb in eastern_conference else "West"


    def check_valid_season(self, season: str) -> str:
        """Check if the provided season is valid and return formatted season."""
        if not season:
            season = self.current_season
        elif not season.isdigit():
            raise ValueError(f"Invalid season format: {season}. Expected a year in 'YYYY' format.")
        elif int(season) < 1980 or int(season) > datetime.now().year:
            raise ValueError(f"Invalid season year: {season}. Year must be between 1980 and {datetime.now().year}.")
        else:
            season = f"{season}-{str(int(season)+1)[-2:]}"
        
        return season

    def get_all_teams(self) -> pd.DataFrame:
        try:
            all_teams = teams.get_teams()
            teams_df = pd.DataFrame(all_teams)
            return teams_df.to_json(orient="records")
        except Exception as e:
            print(f"Error retrieving all teams: {e}")
            raise e
    
    def get_team_details(self) -> Dict:
        try:
            return self.team_details
        except Exception as e:
            print(f"Error retrieving team details for abbreviation {self.team_abbreviation}: {e}")
            raise e
    
    def get_team_details_by_id(self, team_id:int) -> Dict:
        try:
            team_details = teams.find_team_name_by_id(team_id)
            return team_details
        except Exception as e:
            print(f"Error retrieving team details for ID {team_id}: {e}")
            raise e
        
    def get_team_details_by_name(self, team_name:str) -> Dict:
        try:
            team_details = teams.find_teams_by_full_name(team_name)[0]
            return team_details
        except Exception as e:
            print(f"Error retrieving team details for name {team_name}: {e}")
            raise e

    def get_team_roster_per_season(self, season:str = None) -> pd.DataFrame:
        try:
            season = self.check_valid_season(season)

            team_id = self.team_details["id"]
            roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
            roster_data = roster.get_data_frames()[0]
            roster_data = roster_data[["PLAYER", "NUM", "POSITION", "HEIGHT", "WEIGHT", "BIRTH_DATE", "AGE", "EXP", "SCHOOL"]]
            return roster_data
        except Exception as e:
            print(f"Error retrieving roster for team {self.team_details['abbreviation']} in season {season}: {e}")
            raise e

    def get_team_games_played_per_season(self, season:str = None, last_n_games:int = 5) -> pd.DataFrame:
        season = self.check_valid_season(season)
        team_details = self.team_details
        team_id = team_details["id"]
        game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
        game_log = game_log.get_data_frames()[0]

        game_log = game_log[['Game_ID', 'GAME_DATE', 'MATCHUP', 'WL']]
        return game_log[:last_n_games]
    
    def get_team_league_standing(self, season:str = None) -> pd.DataFrame:
        season = self.check_valid_season(season)
        standings = get_current_standings(conference=self.team_details['conference']).reset_index()
        index = int(standings[standings['TeamName'] == self.team_details['nickname']].index[0]) + 1

        return index
    

    def get_team_full_info(self, season:str = None):
        pass



        
    

In [85]:
Teams('LAC').get_team_games_played_per_season(last_n_games=5)

,Game_ID,GAME_DATE,MATCHUP,WL
0,0022500059,"NOV 25, 2025",LAC @ LAL,L
1,0022500278,"NOV 23, 2025",LAC @ CLE,L
2,0022500268,"NOV 22, 2025",LAC @ CHA,W
3,0022500264,"NOV 20, 2025",LAC @ ORL,L
4,0022500243,"NOV 17, 2025",LAC @ PHI,L


In [88]:
#Teams().get_all_teams()
#Teams().get_team_details("LAL")
#print(Teams().get_team_roster_per_season("LAL").columns)
#Teams().get_team_roster_per_season("LAL", season=None)
#Teams("LAL").get_team_games_played_per_season(season=None, last_n_games = 5)

lakers = Teams(team_abbreviation="DEN")

#lakers.get_team_league_standing(season="2023")

team_data = lakers.get_team_details()


team_data['season'] = get_current_season()
team_data['standing'] = lakers.get_team_league_standing()
team_data['last_games'] = lakers.get_team_games_played_per_season().to_dict(orient="records")
team_data['roster'] = lakers.get_team_roster_per_season().to_dict(orient="records")

#print(team_data['roster'])

team_data

#Teams().get_team_by_name("Los Angeles Lakers")
#Teams().get_team_details_by_id(1610612747)
#Teams().get_team_details_by_abbreviation("LAL")


{'id': 1610612743,
 'full_name': 'Denver Nuggets',
 'abbreviation': 'DEN',
 'nickname': 'Nuggets',
 'city': 'Denver',
 'state': 'Colorado',
 'year_founded': 1976,
 'conference': 'West',
 'season': '2025-26',
 'standing': 3,
 'last_games': [{'Game_ID': '0022500287',
   'GAME_DATE': 'NOV 24, 2025',
   'MATCHUP': 'DEN @ MEM',
   'WL': 'W'},
  {'Game_ID': '0022500274',
   'GAME_DATE': 'NOV 22, 2025',
   'MATCHUP': 'DEN vs. SAC',
   'WL': 'L'},
  {'Game_ID': '0022500054',
   'GAME_DATE': 'NOV 21, 2025',
   'MATCHUP': 'DEN @ HOU',
   'WL': 'W'},
  {'Game_ID': '0022500260',
   'GAME_DATE': 'NOV 19, 2025',
   'MATCHUP': 'DEN @ NOP',
   'WL': 'W'},
  {'Game_ID': '0022500248',
   'GAME_DATE': 'NOV 17, 2025',
   'MATCHUP': 'DEN vs. CHI',
   'WL': 'L'}],
 'roster': [{'PLAYER': 'Christian Braun',
   'NUM': '0',
   'POSITION': 'G',
   'HEIGHT': '6-6',
   'WEIGHT': '220',
   'BIRTH_DATE': 'APR 17, 2001',
   'AGE': 24.0,
   'EXP': '3',
   'SCHOOL': 'Kansas'},
  {'PLAYER': 'Curtis Jones',
   'NUM': '1'

In [ ]:
leaderboard = Teams().get_current_standings(conference='East')

In [ ]:
leaderboard[leaderboard['Conference'] == 'West']

,TeamID,TeamCity,TeamName,Conference,WINS,LOSSES,PlayoffRank,Record,HOME,ROAD,L10
0,1610612760,Oklahoma City,Thunder,West,18,1,1,18-1,9-0,9-1,10-0
3,1610612747,Los Angeles,Lakers,West,13,4,2,13-4,5-2,8-2,8-2
5,1610612743,Denver,Nuggets,West,13,4,3,13-4,6-2,7-2,8-2
6,1610612745,Houston,Rockets,West,12,4,4,12-4,5-2,7-2,8-2
9,1610612759,San Antonio,Spurs,West,12,5,5,12-5,8-2,4-3,7-3
10,1610612756,Phoenix,Suns,West,12,7,6,12-7,8-3,4-4,8-2
13,1610612750,Minnesota,Timberwolves,West,10,8,7,10-8,5-3,5-5,6-4
15,1610612744,Golden State,Warriors,West,10,10,8,10-10,6-2,4-8,5-5
17,1610612757,Portland,Trail Blazers,West,8,11,9,8-11,3-5,5-6,3-7
19,1610612763,Memphis,Grizzlies,West,7,12,10,7-12,4-6,3-6,4-6
